In [73]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
import matplotlib.pyplot as plt

import pandas as pd

import math

# tells matplotlib to embed plots within the notebook
%matplotlib inline

# Numerical Data only test

### Import the training dataset and Feature engineering

In [74]:
# import training CSV
fname = './data/train.csv'
# use pandas to read the CSV properly of all different types
data_pd = pd.read_csv(fname)
# convert to a numpy array
data = np.array(data_pd)
# remove categorical data - force cast attempt then remove failures
cols_to_remove = []
for i in range(data.shape[1]):
    try:
        data[:,i] = data[:,i].astype(float)
    except:
        cols_to_remove.append(i)

data = np.delete(data, cols_to_remove, axis=1)
# remove rows with nan values
rows_to_remove = []
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if math.isnan(data[i,j]):
            rows_to_remove.append(i)
            break

data = np.delete(data, rows_to_remove, axis=0)

# remove outliers with standard deviation TODO()

# take the actual prices and add them to results y
y = data[:, data.shape[1] - 1]
# remove prices column from data
data = np.delete(data, data.shape[1] - 1, 1)
data = data.astype(float)
print("data shape", data.shape)
print("Y shape:", y.shape)

pd.DataFrame(data)

data shape (1121, 37)
Y shape: (1121,)


,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,1.0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,...,548.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0
1,2.0,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,...,460.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0
2,3.0,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,...,608.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0
3,4.0,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,...,642.0,0.0,35.0,272.0,0.0,0.0,0.0,0.0,2.0,2006.0
4,5.0,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,...,836.0,192.0,84.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,1456.0,60.0,62.0,7917.0,6.0,5.0,1999.0,2000.0,0.0,0.0,...,460.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,8.0,2007.0
1117,1457.0,20.0,85.0,13175.0,6.0,6.0,1978.0,1988.0,119.0,790.0,...,500.0,349.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2010.0
1118,1458.0,70.0,66.0,9042.0,7.0,9.0,1941.0,2006.0,0.0,275.0,...,252.0,0.0,60.0,0.0,0.0,0.0,0.0,2500.0,5.0,2010.0
1119,1459.0,20.0,68.0,9717.0,5.0,6.0,1950.0,1996.0,0.0,49.0,...,240.0,366.0,0.0,112.0,0.0,0.0,0.0,0.0,4.0,2010.0


In [75]:
# [CHECKPOINT 5][5 points]
def  feature_normalization(X):
    """
    Normalizes the features in X. returns a normalized version of X where
    the mean value of each feature is 0 and the standard deviation
    is 1. This is often a good preprocessing step to do when working with
    learning algorithms.
    
    Parameters
    ----------
    X : array_like
        The dataset of shape (m x n).
    
    Returns
    -------
    X_norm : array_like
        The normalized dataset of shape (m x n).
    
    Instructions
    ------------
    First, for each feature dimension, compute the mean of the feature
    and subtract it from the dataset, storing the mean value in mu. 
    Next, compute the  standard deviation of each feature and divide
    each feature by it's standard deviation, storing the standard deviation 
    in sigma. 
    
    Note that X is a matrix where each column is a feature and each row is
    an example. You needto perform the normalization separately for each feature. 
    
    Hint
    ----
    You might find the 'np.mean' and 'np.std' functions useful.
    """
    mu = []
    sigma = []
    X_norm = np.copy(X)
    
    col_to_remove = []
    for i in range(X_norm.shape[1]):
        mu.append(np.mean(X_norm[:,i]))
        sigma.append(np.std(X_norm[:,i]))
    X_norm = np.delete(X_norm, col_to_remove, axis=1)

    # =========================== YOUR CODE HERE =====================
    for i in range(X_norm.shape[1]):
        if np.std(X_norm[:,i]) != 0:
            X_norm[:,i] = (X_norm[:,i] - np.mean(X_norm[:,i])) / np.std(X_norm[:,i])
        else:
            X_norm[:,i] = (X_norm[:,i] - np.mean(X_norm[:,i])) / 0.01
    
    
    # ================================================================
    return X_norm, mu, sigma

### Normal Equation and Polynomial Regression Functions

In [76]:
data, mu, sigma = feature_normalization(data)
# replace example # column with 1's intercept column
data[:,0] = np.ones(data.shape[0])
pd.DataFrame(data)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,1.0,0.092295,-0.233570,-0.205885,0.570704,-0.525499,0.992930,0.823953,0.462009,0.571581,...,0.235641,-0.760257,0.231036,-0.356622,-0.11253,-0.278676,-0.072999,-0.141407,-1.615345,0.153084
1,1.0,-0.865696,0.384834,-0.064358,-0.153825,2.284122,0.120665,-0.460746,-0.572748,1.152559,...,-0.224712,1.686090,-0.716739,-0.356622,-0.11253,-0.278676,-0.072999,-0.141407,-0.498715,-0.596291
2,1.0,0.092295,-0.109889,0.138702,0.570704,-0.525499,0.928317,0.776371,0.282510,0.101672,...,0.549518,-0.760257,-0.064173,-0.356622,-0.11253,-0.278676,-0.072999,-0.141407,0.990125,0.153084
3,1.0,0.331793,-0.439705,-0.070512,0.570704,-0.525499,-1.850006,-0.746235,-0.572748,-0.475034,...,0.727382,-0.760257,-0.172934,4.083851,-0.11253,-0.278676,-0.072999,-0.141407,-1.615345,-1.345665
4,1.0,0.092295,0.549742,0.509132,1.295234,-0.525499,0.896011,0.681208,1.275032,0.462647,...,1.742250,0.815913,0.588393,-0.356622,-0.11253,-0.278676,-0.072999,-0.141407,2.106755,0.153084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,1.0,0.092295,-0.357251,-0.271480,-0.153825,-0.525499,0.863705,0.681208,-0.572748,-0.936399,...,-0.224712,-0.760257,-0.095247,-0.356622,-0.11253,-0.278676,-0.072999,-0.141407,0.617915,-0.596291
1117,1.0,-0.865696,0.590968,0.375605,-0.153825,0.411042,0.185277,0.110231,0.055497,0.751000,...,-0.015461,2.104761,-0.716739,-0.356622,-0.11253,-0.278676,-0.072999,-0.141407,-1.615345,1.651832
1118,1.0,0.331793,-0.192343,-0.133030,0.570704,3.220663,-1.010048,0.966697,-0.572748,-0.349013,...,-1.312819,-0.760257,0.215498,-0.356622,-0.11253,-0.278676,-0.072999,14.947388,-0.498715,1.651832
1119,1.0,-0.865696,-0.109889,-0.049960,-0.878355,0.411042,-0.719293,0.490883,-0.572748,-0.831738,...,-1.375594,2.244317,-0.716739,1.471808,-0.11253,-0.278676,-0.072999,-0.141407,-0.870925,1.651832


### 3.2 Normal equation

We have learned that the closed-form solution to linear regression is

$$ \theta = \left( X^T X\right)^{-1} X^T\vec{y}$$

Using this formula does not require any feature scaling, and you will get an exact solution in one calculation: there is no “loop until convergence” like in gradient descent. 

Hint: Before implement normal_equation function, following example shows how to convert array data to a matrix with a shape of $m \times 1$, and generate polynomial features matrix $[1, X, X^2, X^3, \cdots]$. 

In [81]:
# [CHECKPOINT 2][10 points]
def normal_equation(X, Y, n):
    """
    Computes the closed-form solution to linear regression using the normal equations.
    
    Parameters
    ----------
    X : array_like
        The dataset of shape (m, ).
    
    Y : array_like
        The value at each data point. A vector of shape (m, ).
        
    n : the order of polynomial regression model
        Remember the number of features will be n+1.
    
    Returns
    -------
    theta : array_like
        Estimated polynomial regression parameters. A vector of shape (n+1, ).
    
    Instructions
    ------------
    Complete the code to compute the closed form solution to linear
    regression and put the result in theta.
    
    Hint
    ----
    Look up the function `np.linalg.pinv` for computing matrix inverse.
    """
    #print("Normal equation start")
    #print("n: ", n)
    theta = np.zeros(n+1)
    #print("theta: ", theta)
    
    # ===================== YOUR CODE HERE ============================
    X_t = np.transpose(X)
    theta = np.matmul(X_t, X)
    theta = np.linalg.pinv(theta)
    tmp = np.matmul(X_t, y)
    theta = np.matmul(theta, tmp)
    # =================================================================
    return theta.flatten()

In [82]:
# [CHECKPOINT 3][5 points]
def polynomial_deploy(X, theta):
    """
    Computes the polynomial regression prediction for data X.
    
    Parameters
    ----------
    X : array_like
        The input data. A vector of shape (m, ).
    
    theta : array_like
        Polynomial regression parameters. A vector of shape (n+1, ).
    
    Returns
    -------
    Y : array_like
        Polynomial prediction. A vector of shape (m, ).
    
    """
    #print("Begin polynomial deploy")
    n = theta.size
    Y = np.array([])
    
    # ===================== YOUR CODE HERE ============================

    theta_t = theta.reshape(n, 1)

    #Y = np.matmul(np.transpose(theta), X)
    Y = np.matmul(X, theta_t)
            
    #print("X poly size: ", X_poly.shape)
    
    # ===================== YOUR CODE HERE ============================
    return Y.flatten()

In [83]:
#MSE helper func
def MSE_func(m, hypothesis, actual):
    MSE = 0
    for i in range(m):
            MSE += (hypothesis[i] - actual[i]) ** 2
    return MSE

### Polynomial Function Predictions and MSE

In [92]:
from sklearn import metrics

n = data.shape[1]
m = data.shape[0]
#print("n:", n)
X_data = data
theta = normal_equation(X_data, y, n)
#print("theta x set: ", theta)
prediction = polynomial_deploy(X_data, theta)
#print(prediction.shape)
MSE = MSE_func(X_data.shape[0], prediction, y)
print("MSE:", MSE)
print('VarScore:',metrics.explained_variance_score(y,prediction))

MSE: 1469655140896.9622
VarScore: 0.8095197151979675
